# Access Scores

## Calculating the Access Score
---

<span style="color:LimeGreen"> **Specifications for the HSSA Model:**  

HSSAModel( ):  
- Network_type ('drive' or 'walk')
- source_filename
- dest_filename
- sp_matrix_filename (None implies that the user hasn't previously generated a distance matrix; thus, calculating it on the fly.)  
- decay_function ('linear', 'root', 'logit')
- limit_categories (None is defined as default, but user can delimit on particular categories of interest.)  
- upper=30 (can change default) is the time (in minutes) in which the origin and destinations are considered to be out of range of each other. Thus, if the travel time distance exceeds 30 minutes, the score will be zero.
- custom_threshold=40 (change binning threshold specifying it in calculate() function. Example: model_name.calculate(custom_threshold=40)

Spatial areal units with no population are denominated as NA (>>> WRONG - CHANGE after Logan fixes) while spatial areal units beyond the 30 minute buffer are denominated as zeros.

In **model_name.calculate( )** specify the following parameters:  
* normalize=True (if want standarized and unstandarizes score outputs)
* use_target=False (neglect target as a weight) >>> will be deprecated soon
* custom_weight_dict=None (set dictionary of weights if user has specific loadings of interest)  
* largest_weights_first=True (set dictionary or default weights as the largest ones first)

Dictionary weights should be set as the following example:  
my_dict = {'Hospitals' : [200, 300,299, 250],  
                'Federally Qualified Health Centers ' : [60,70,70,80,90,100],  
                'School-Based Health Centers': [40,20,10],  
                'All Free Health Clinics': [100,20,1,5]}

In [ ]:
from CommunityAnalytics import *

In [ ]:
#the last already generated a shortest path matrix, so we can reuse that instead of regenerating it

access = AccessModel(network_type='walk',
                     sp_matrix_filename='data/walk_nets_tracts2010.csv',
                     source_filename='data/tracts2010.csv',
                     dest_filename='data/nets_chicago2.csv',
                     decay_function='logit')

In [ ]:
#Example of density and variety weights:
dict = {
"Hospitals": [10,10,10,10,10],
"Federally Qualified Health Centers": [8, 7, 6, 5, 4],
"School-Based Health Centers": [7, 7, 6, 6, 5],
"All Free Health Clinics": [5, 5, 5, 4, 4],
"Other Health Providers": [4,3,2,1,1]
}

In [ ]:
access.calculate(custom_threshold=40,
                 normalize=True,
                 custom_weight_dict=dict,
                 largest_weights_first=True)

By constructing the access model, we can do several things:
- Write to csv
- Aggregate
- Get count of nearby destinations metric
- Get time to closest destination metric

In [ ]:
access.results

### Aggregation

In [ ]:
#For community areas
access.write_aggregate('access')

## Count of Nearby Destinations

In [ ]:
#Use .n_dests_in_range to estimate the number of facilities by category within a buffer (time defined in upper)
access.n_dests_in_range.to_csv('data/nd_health_tracts.csv')

In [ ]:
access.n_dests_in_range

## Time to Closest Destination

In [ ]:
#Use .near_nbr to create the time to the nearest neighbor within a buffer (time defined in upper)
access.near_nbr.to_csv('data/nn_health_tracts.csv')

In [ ]:
access.near_nbr

# Coverage Score

## Calculating the Per Capita Access Score
---


**Specifications for PCSpend Model:**  
PCSpendModel( ):  
- Network_type ('drive' or 'walk')
- source_filename  
- dest_filename
- sp_matrix_filename (None implies that the user hasn't previously generated a distance matrix; thus, calculating it on the fly.)  
- limit_categories (None is defined as default, but user can focus only on particular categories of interest.)  


Specifications:
- Target or amenity value (>>>where specify this? Should we still have the option to carry it out in the HSSA? Should we specify that there?): The establishment will be weighted depending of the size of the establishment. Size can be measured differently (in this case, it's the size of the health facility, but it could also be the number of patients it accomodates, etc.)   
- Category of interest

Functions within the PCSpend Model class (use as name_of_model.function()):  
- calculate () 
- calculate_single  (subset)
- get_aggregate (aggregate_type) Aggregation can be by ('population') or ('pcspend') score for the community level. If the user wants to modify the level of aggregation, change 'community' with other spatial unit of analysis in code.  
- write_aggregate (aggregate_type, filename='name')
- plot_aggregate  (plot_type='pcspend', include_vendors=True, focus_column='all_categories')
- plot_cdf  (title='Per Capita Spending')
- write_csv (filename='name') 

In [ ]:
coverage = CoverageModel(network_type='walk',
                         sp_matrix_filename='data/walk_asym_health_tracts.csv',
                         source_filename='data/tracts2010.csv',
                         dest_filename='data/marynia_health.csv')

In [ ]:
coverage.calculate()

### Aggregation

In [ ]:
coverage._get_aggregate('coverage')

In [ ]:
coverage.write_aggregate('coverage')

In [ ]:
coverage.agg_area_cat()

### Subset of categories

In [ ]:
# Estimate Per Capita Access Score for walking
coverage_hosp = PCSpendModel(network_type='walk',
                        sp_matrix_filename='data/walk_asym_health.csv',
                        source_filename='resources/LEHD_blocks.csv',
                        dest_filename='resources/marynia_health.csv',
                        upper=30,
                        limit_categories=set(['Federally Qualified Health Centers', 'Hospitals', 'School-Based Health Centers']))